## Setting up a new sensor as science-jubilee toolhead

This notebook will walk through setting up a new sensor!

## Introduction

Making a sensor tool on Jubilee is easy - grab a sensor, bolt it down onto a tool head, and use it!

We are using a modular approach to integrate sensor(s). Each sensor is wired to its own microcontroller node that is then connected to a laptop/raspberry pi for orchestration within the science-jubilee framework.

First, program the node so that it communicates with science-jubilee via serial. The node is a custom pcb with Seeed Xiao as its brain and has a I2C connector.

![Seeed microcontroller](images/microcontroller.png)

### Installing CircuitPython on ESP32 (OPTIONAL EXERCISE)

***The microcontrollers provided have been flashed with CircuitPython; this section is for documentation purpose.***

There is no Xiao ESP32S3-specific CircuitPython .bin firmware as of Jan 21 2024, so using [Espressif - ESP32-S3-DevKitM-1-N8](https://circuitpython.org/board/espressif_esp32s3_devkitm_1_n8/) is currently the best way to do it. Download the latest .bin from the link.

Adafruit released CircuitPython for Xiao ESP32S3 in Jan 2025 so you can also install [this](https://circuitpython.org/board/seeed_xiao_esp32s3_sense/).

### Installing the Firmware

This must be done over serial. We are using `esptool`, but there is also guidance on how to use the web serial browser based tool (easy method, but requires a Chrome webbrowser).

In the terminal, install esptool using pip if you haven't yet. More instruction can be found [here](https://docs.espressif.com/projects/esptool/en/latest/esp32/index.html#quick-start)

```
pip install esptool
```

Find all serial devices by running `ls /dev/tty.*`. On Mac it is usually `/dev/tty.usbmodem******`.

Once you have esptool installed, you will first want to erase the flash on your ESP32 board, it's also a great way to determine that you are able to connect. Change the serial address to the one that shows up on your terminal. I'm using ESP32S3 so the chip argument is also changed.

`esptool.py --chip esp32s3 --port /dev/tty.usbmodem142401 erase_flash`

If you're not 100% sure which ESP32 chip you have, you can just specify the port: `esptool.py --port /dev/tty.usbmodem****** erase_flash`

Once the flash is successfully erased, you can load your firmware `.bin` file by running something similar to the command below. Make sure you update the following:

- The port to match the serial port to which your board is connected, i.e. change /dev/tty.usbsmodem****** to match your connection.

- The **firmware.bin** file name to match the firmware you downloaded, i.e. change firmware.bin to something like `adafruit-circuitpython-espressif_esp32s3_devkitm_1_n8-en_US-8.2.9.bin`. 

- This command assumes you will be loading it to address 0x0, as we do for circuitpython. This may not be true for your firmware! check the documentation for the offset if necessary.

Load the firmware using something similar to the following command, with the above changes:

`esptool.py --port /dev/tty.usbmodem142401 write_flash -z 0x0 adafruit-circuitpython-espressif_esp32s3_devkitm_1_n8-en_US-9.2.0.bin`

You will then see a flash storage unit named `CIRCUITPY` popping up.

### Adding CircuitPython firmware

In the `CIRCUITPY` device there should be a `code.py`, a `boot.py`, a `boot.out` (used for debugging only), and a directory `lib`.

The [basic code](Seeed_files/code.py) has a `blink` function which blinks the on-board LED. [`boot.py`](Seeed_files/boot.py) is used for initializing sensor communication. `code.py` can be shared among microcontrollers while `boot.py` should contain sensor-specific setup code.

We want to setup AS7314 spectrometer. It comes with its own CircuitPython library, which needs to be added to the `lib` folder first.

***Alternatively***, copy the entire `lib` folder containing all current libraries available for CircuitPython. Adafruit maintains and updates these libraries and the latest bundle can be found [here](https://circuitpython.org/libraries) (note that there might be compatibility issue).

#### Step 1

Copy [everything in this folder](Seeed_files) into CIRCUITPY.

#### Step 2

Add sensor-specific code to boot. Paste the following code into `boot.py`.

```python
# function to initialize the sensor on I2C
def initialize_peripherals():
    i2c = busio.I2C(board.IO6, board.IO5) # initialize I2C
    sensor = AS7341(i2c) # initialize sensor
    return sensor

# Initialize
sensor = initialize_peripherals() # initialize the sensor which will be imported on the code.py file
```
#### Step 3

Add sensor-specific function to the class in the main program. Paste the following code following the `blink` example code; make sure the indentation is correct.

```python
    def case_spec(self, param=None):
        sensor.led_current = int(param) # set LED current using the input argument
        time.sleep(1) # let it illuminate for 1 second
        # read the sensor data
        data = { # sensor data stored as a dictionary
            '415': sensor.channel_415nm,
            '445': sensor.channel_445nm,
            '480': sensor.channel_480nm,
            '515': sensor.channel_515nm,
            '555': sensor.channel_555nm,
            '590': sensor.channel_590nm,
            '630': sensor.channel_630nm,
            '680': sensor.channel_680nm,
        }
        print(data) # print the sensor data via serial back to science jubilee. On science jubilee, data is received as a str
        sensor.led_current = 0 #turn off LED
```

#### Step 4

When working with a new tool, you will need to add tool definition to Science Jubilee. Check [extending-science-jubilee](../extending-science-jubilee/readme.md) for more detail!

The spectral sensor is already added to the science-jubilee library.

#### Step 5

Use it!

In [ ]:
# import science jybilee
from science_jubilee.Machine import Machine
# import the new tool
from science_jubilee.tools.AS7341 import AS7341
# import json for data handling
import json

In [6]:
# Connect to the machine
m = Machine(address='192.168.1.2')

In [ ]:
# define the tool
spec = AS7341(index = 0, name = "AS7341 spectral sensor", config = "AS7341")
# load the tool
m.load_tool(spec)

In [4]:
# pick up the spectral sensor tool!
m.pickup_tool(spec)

# sensors are controlled by and communicated via nodes! and nodes need to be connected to the pi first ->
spec.connect_seeed() 
# we only have one node now so there is no need for an index 
# but we will probably get to multiple sensors later

# blink the connected node! the argument in the blink function is the duration of blinking in second
spec.blink("1")

In [ ]:
# yay it blinks! now let's take a measurement
# optional: move the sensor around to be closer to the object you want to measure
m.move_to(z=0)
# moving it below z=0 is possible but dangerous!
# the argument is the current of the LEDs surrounding the spectral sensor
# range: 0 - 40 mA
spec.measure_spectrum(3)

In [10]:
# Misc.
# Dropoff tool!
m.park_tool()